In [ ]:
import json
import os
import os
import shutil

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
from ultralytics import YOLO

In [ ]:


# Define the directories
label_directory = '/media/avalocal/T9/harsha/datasets/data_copy/labels/test'
image_directory = '/media/avalocal/T9/harsha/datasets/data_copy/images/test'
output_base_directory = '/media/avalocal/T9/harsha/datasets/speed_limit_data/test'

# Define the output subdirectories
output_image_directory = os.path.join(output_base_directory, 'images')
output_label_directory = os.path.join(output_base_directory, 'labels')

# Define the set of first entries to filter out
filter_set = {80, 81, 82, 83, 84, 85, 86, 87, 88, 89}

# Create the output directories if they don't exist
os.makedirs(output_image_directory, exist_ok=True)
os.makedirs(output_label_directory, exist_ok=True)

def filter_files(label_directory, filter_set):
    image_files_to_filter = []
    label_files_to_filter = []
    
    for filename in os.listdir(label_directory):
        if filename.endswith('.txt'):
            filepath = os.path.join(label_directory, filename)
            with open(filepath, 'r') as file:
                lines = file.readlines()
                # Check each line in the file
                for line in lines:
                    first_entry = int(line.split()[0])
                    if first_entry in filter_set:
                        image_filename = filename.replace('.txt', '.jpg')
                        image_files_to_filter.append(image_filename)
                        label_files_to_filter.append(filename)
                        break

    return image_files_to_filter, label_files_to_filter

def copy_filtered_files(files_to_filter, source_directory, output_directory):
    for file in files_to_filter:
        src_path = os.path.join(source_directory, file)
        dst_path = os.path.join(output_directory, file)
        if os.path.exists(src_path):
            shutil.copy(src_path, dst_path)
            print(f"Copied {file} to {output_directory}")
        else:
            print(f"File {file} does not exist in {source_directory}")

# Run the filtering function and get the list of image and label files to filter
image_files_to_filter, label_files_to_filter = filter_files(label_directory, filter_set)

# Copy the filtered image files to the new image directory
copy_filtered_files(image_files_to_filter, image_directory, output_image_directory)

# Copy the filtered label files to the new label directory
copy_filtered_files(label_files_to_filter, label_directory, output_label_directory)

# Print the list of copied image and label files
print("Filtered and copied image files:", image_files_to_filter)
print("Filtered and copied label files:", label_files_to_filter)


In [ ]:
import os

# counting the number of occurences of each class in the dataset
# Define the class names dictionary
class_names = {
    80: "Speed limit: 70",
    81: "Speed limit: 75",
    82: "Speed limit: 30",
    83: "Speed limit: 35",
    84: "Speed limit: 40",
    85: "Speed limit: 45",
    86: "Speed limit: 50",
    87: "Speed limit: 55",
    88: "Speed limit: 60",
    89: "Speed limit: 65",
    90: "Stop",
    91: "Red",
    92: "Yellow",
    93: "Green",
}

# Initialize a dictionary to count occurrences of each class ID
class_counts = {class_id: 0 for class_id in class_names.keys()}

# Path to the folder containing the txt files
folder_path = "/media/avalocal/T9/harsha/datasets/speed_limit_data/train/labels"

# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r") as file:
            lines = file.readlines()
            for line in lines:
                class_id = int(line.split()[0])
                if class_id in class_counts:
                    class_counts[class_id] += 1

# Print the number of occurrences of each class
for class_id, count in class_counts.items():
    class_name = class_names[class_id]
    print(f"{class_name}: {count}")

In [ ]:
import os
import shutil
import random
from collections import defaultdict

# Define the source and destination directories
source_directory = '/media/avalocal/T9/harsha/datasets/speed_limit_data'
destination_directory = '/media/avalocal/T9/harsha/datasets/speed_limit_data_all_labels_mini'

# Number of samples per label
samples_per_label = 500

# List of labels
label_list = ['80', '81', '82', '83', '84', '85', '86', '87', '88', '89']

# Create the destination directory and its subdirectories if they don't exist
os.makedirs(destination_directory, exist_ok=True)
os.makedirs(os.path.join(destination_directory, 'train/images'), exist_ok=True)
os.makedirs(os.path.join(destination_directory, 'train/labels'), exist_ok=True)
os.makedirs(os.path.join(destination_directory, 'val/images'), exist_ok=True)
os.makedirs(os.path.join(destination_directory, 'val/labels'), exist_ok=True)

def get_images_by_label(source_labels_dir):
    label_to_images = defaultdict(list)
    for label_file in os.listdir(source_labels_dir):
        if label_file.endswith('.txt'):
            with open(os.path.join(source_labels_dir, label_file), 'r') as f:
                for line in f:
                    label = line.strip().split(' ')[0]
                    if label in label_list:
                        image_file = label_file.replace('.txt', '.jpg')
                        label_to_images[label].append(image_file)
    return label_to_images

def copy_samples(source_images_dir, source_labels_dir, dest_images_dir, dest_labels_dir, label_to_images, num_samples):
    for label in label_list:
        image_files = label_to_images[label]
        # Randomly sample the required number of images
        sampled_images = random.sample(image_files, min(num_samples, len(image_files)))
        for image_file in sampled_images:
            # Define source and destination paths for images
            src_image_path = os.path.join(source_images_dir, image_file)
            dst_image_path = os.path.join(dest_images_dir, image_file)
            
            # Copy image file
            try:
                shutil.copy(src_image_path, dst_image_path)
            except:
                continue
            
            # Define source and destination paths for corresponding label files
            label_file = image_file.replace('.jpg', '.txt')
            src_label_path = os.path.join(source_labels_dir, label_file)
            dst_label_path = os.path.join(dest_labels_dir, label_file)
            
            # Copy label file
            shutil.copy(src_label_path, dst_label_path)
            
            print(f"Copied {image_file} and {label_file} to {dest_images_dir} and {dest_labels_dir}")

# Get images by label for train set
train_label_to_images = get_images_by_label(os.path.join(source_directory, 'train/labels'))
# Copy samples from train set
copy_samples(
    source_images_dir=os.path.join(source_directory, 'train/images'),
    source_labels_dir=os.path.join(source_directory, 'train/labels'),
    dest_images_dir=os.path.join(destination_directory, 'train/images'),
    dest_labels_dir=os.path.join(destination_directory, 'train/labels'),
    label_to_images=train_label_to_images,
    num_samples=samples_per_label
)

# Get images by label for val set
val_label_to_images = get_images_by_label(os.path.join(source_directory, 'val/labels'))

# Copy samples from val set
copy_samples(
    source_images_dir=os.path.join(source_directory, 'val/images'),
    source_labels_dir=os.path.join(source_directory, 'val/labels'),
    dest_images_dir=os.path.join(destination_directory, 'val/images'),
    dest_labels_dir=os.path.join(destination_directory, 'val/labels'),
    label_to_images=val_label_to_images,
    num_samples=samples_per_label
)

print("Sampling and copying complete.")


In [ ]:
import os
import shutil
import random

# Define the source and destination directories
source_directory = '/media/avalocal/T9/harsha/datasets/speed_limit_data'
destination_directory = '/media/avalocal/T9/harsha/datasets/speed_limit_data_all_labels_mini'

# Number of samples to copy
train_samples = 5000
val_samples = 2000

# Create the destination directory and its subdirectories if they don't exist
os.makedirs(destination_directory, exist_ok=True)
os.makedirs(os.path.join(destination_directory, 'train/images'), exist_ok=True)
os.makedirs(os.path.join(destination_directory, 'train/labels'), exist_ok=True)
os.makedirs(os.path.join(destination_directory, 'val/images'), exist_ok=True)
os.makedirs(os.path.join(destination_directory, 'val/labels'), exist_ok=True)

def copy_samples(source_images_dir, source_labels_dir, dest_images_dir, dest_labels_dir, num_samples):
    image_files = os.listdir(source_images_dir)
    image_files = [f for f in image_files if f.endswith('.jpg')]
    
    # Randomly sample the required number of images
    sampled_images = random.sample(image_files, num_samples)
    
    for image_file in sampled_images:
        # Define source and destination paths for images
        src_image_path = os.path.join(source_images_dir, image_file)
        dst_image_path = os.path.join(dest_images_dir, image_file)
        
        # Copy image file
        shutil.copy(src_image_path, dst_image_path)
        
        # Define source and destination paths for corresponding label files
        label_file = image_file.replace('.jpg', '.txt')
        src_label_path = os.path.join(source_labels_dir, label_file)
        dst_label_path = os.path.join(dest_labels_dir, label_file)
        
        # Copy label file
        shutil.copy(src_label_path, dst_label_path)
        
        print(f"Copied {image_file} and {label_file} to {dest_images_dir} and {dest_labels_dir}")

# Copy samples from train set
copy_samples(
    source_images_dir=os.path.join(source_directory, 'train/images'),
    source_labels_dir=os.path.join(source_directory, 'train/labels'),
    dest_images_dir=os.path.join(destination_directory, 'train/images'),
    dest_labels_dir=os.path.join(destination_directory, 'train/labels'),
    num_samples=train_samples
)

# Copy samples from val set
copy_samples(
    source_images_dir=os.path.join(source_directory, 'val/images'),
    source_labels_dir=os.path.join(source_directory, 'val/labels'),
    dest_images_dir=os.path.join(destination_directory, 'val/images'),
    dest_labels_dir=os.path.join(destination_directory, 'val/labels'),
    num_samples=val_samples
)

print("Sampling and copying complete.")


In [ ]:
# Testing on test image
# test_dataset_dir = (
#     "/home/avalocal/Documents/yolov9_ros/src/yolov9ros/src/yolov9/cone_images"
# )
test_dataset_dir = '/media/avalocal/T9/harsha/datasets/coco_mini/images/val'
# Get the list of images in the test dataset directory
test_images = os.listdir(test_dataset_dir)

model = YOLO(
    '/home/avalocal/Documents/yolov9_ros/src/yolov9ros/src/runs/detect/train3/weights/best.pt'
)

# for image_name in test_images[0, len(test_images), 100]:
for img_id in range(0, len(test_images), 100):
    # Load the image
    image_path = os.path.join(test_dataset_dir, test_images[img_id])
    image = cv2.imread(image_path)
    image.shape
    detections = model(image)
    result = detections[0]
    class_id = result.boxes.cls.cpu().numpy()  # Class IDs
    confidence = result.boxes.conf.cpu().numpy()  # Confidence scores

    bboxes = np.array(result.boxes.xyxy.cpu(), dtype="int")  # Bounding boxes
    for i, bbox in enumerate(bboxes):
        (x, y, x2, y2) = bbox
        cv2.rectangle(image, (x, y), (x2, y2), (0, 0, 225), 2)
        label = f"{model.names[int(class_id[i])]}: {float('{:.2f}'.format(confidence[i]))}"
        cv2.putText(
            image,
            label,
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 0, 225),
            2,
        )

    # Convert BGR to RGB for displaying with matplotlib
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the image with bounding boxes using matplotlib
    plt.figure(figsize=(8, 6))
    plt.imshow(image_rgb)
    plt.axis("off")
    plt.show()